In [14]:
# Initialise azure machine learning workspace
from azureml import Workspace
ws = Workspace()

# Loads datasets from workspace
index_master_ds = ws.datasets['Index_Master.csv']
index_ds = ws.datasets['Index.csv']
equity_ds = ws.datasets['Equity.csv']

# Cast to dataframes
index_meta = index_master_ds.to_dataframe()
index_df = index_ds.to_dataframe()
equity_df = equity_ds.to_dataframe()

In [16]:
aapl_ds = ws.datasets['AAPL_closing.csv']

In [18]:
print(type(index_ds))
print(type(aapl_ds))

<class 'azureml.SourceDataset'>
<class 'azureml.SourceDataset'>


In [26]:


#aapl = aapl_ds.to_dataframe()
index = index_ds.to_dataframe()
print(dir(aapl_ds))
print()
print(dir(index_ds))

['Location', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_metadata', '_metadata_repr', '_to_dataframe', '_update_from_dataframe', '_update_from_raw_data', '_upload_and_refresh', 'batch', 'category', 'client_version', 'contents_url', 'created_date', 'created_date_ticks', 'culture', 'data_type_id', 'dataset_id', 'description', 'download_location', 'experiment_id', 'family_id', 'is_deprecated', 'is_example', 'is_latest', 'name', 'open', 'owner', 'promoted_from', 'read_as_binary', 'read_as_text', 'resource_upload_id', 'schema_end_point', 'schema_status', 'service_version', 'size', 'source_origin', 'update_from_dataframe', 'update_from_raw_data', 'uploaded_from_filename', 'visualize_end_point', 'workspace']

['Locat

## Pre-processing finanical data
1. Fill the missing stock prices (i.e. Saturdays/Sundays)
2. Compute log returns
3. Pick out outliers, fill them back in with interpolation

The suite of main functions operates on a dataframe of one ticker. Use the getTicker() function to preprocess the main dataframe into a subset of one ticker

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from scipy.stats import norm

def getTicker(df, ticker):
    return df[df['Ticker'] == ticker]

def fillMissingData(ticker_df):
    # parse date string to date objects
    ticker_dates = pd.to_datetime(ticker_df['Date'], format='%Y-%m-%d').tolist()
    ticker_price = ticker_df['Close'].tolist()
    
    for t1 in ticker_dates[0:-1]:
        
        # update the variables every iteration to make sure it they are consistent
        t1_i = ticker_dates.index(t1)
        t2 = ticker_dates[t1_i + 1]
        day_diff = (t2 - t1).days
        
        if day_diff > 1:
            for i in range(day_diff-1):
                # inserts the interpolated price from the large to small (in case of multiple days gap)
                ticker_dates.insert(t1_i+1, t1 + timedelta(days=day_diff-i-1))
                ticker_price.insert(t1_i+1, ticker_price[t1_i] + (ticker_price[t1_i + 1] - ticker_price[t1_i]) * (day_diff-i-1) / day_diff)
    
    df = pd.DataFrame({
            'Date': ticker_dates,
            'Close': ticker_price
        })
    return df

def getLogReturn(ticker_df, tau):
    ticker_df = ticker_df.as_matrix()
    lr = np.log(ticker_df[tau:]/ticker_df[0:-tau])
    return lr

def findOutliers(lr):
    mu, sigma = norm.fit(lr)
    outliers = np.logical_or(lr < mu-3*sigma, lr > mu+3*sigma)
    return [i for i in range(len(outliers)) if outliers[i] == True]

# assumes that outliers do not sit next to each other (to make my life easier)
def removeOutliers(lr, outliers_index):
    outliers_index = np.array(outliers_index)

    slid_forth = outliers_index - 1
    slid_back = outliers_index + 1

    lr[outliers_index] = (lr[slid_forth] + lr[slid_back] )/2
    return lr

# Usage demo

In [27]:
ticker = 'AAPL US Equity'
tau = 1
df = equity_df

ticker_df = getTicker(df, ticker)
ticker_df = fillMissingData(ticker_df)
lr = getLogReturn(ticker_df['Close'], tau)
outliers = findOutliers(lr)
lr = removeOutliers(lr, outliers)
pd.DataFrame({
    'timestamp': ticker_df['Date'][:-1],
    'price': lr
})
unix_timestamp = ticker_df['Date'][:-1].astype(np.int64) // 10**9
unix_timestamp

0       1167782400
1       1167868800
2       1167955200
3       1168041600
4       1168128000
5       1168214400
6       1168300800
7       1168387200
8       1168473600
9       1168560000
10      1168646400
11      1168732800
12      1168819200
13      1168905600
14      1168992000
15      1169078400
16      1169164800
17      1169251200
18      1169337600
19      1169424000
20      1169510400
21      1169596800
22      1169683200
23      1169769600
24      1169856000
25      1169942400
26      1170028800
27      1170115200
28      1170201600
29      1170288000
           ...    
3687    1486339200
3688    1486425600
3689    1486512000
3690    1486598400
3691    1486684800
3692    1486771200
3693    1486857600
3694    1486944000
3695    1487030400
3696    1487116800
3697    1487203200
3698    1487289600
3699    1487376000
3700    1487462400
3701    1487548800
3702    1487635200
3703    1487721600
3704    1487808000
3705    1487894400
3706    1487980800
3707    1488067200
3708    1488

Nothing to see down there

In [5]:
# Just a random try stuff out box

import pandas as pd
from datetime import datetime
from datetime import date
from datetime import timedelta

d = date(5, 12, 17)
d = datetime.strptime('01-12-15', '%d-%m-%y')

date = pd.DataFrame(['28-05-07', '29-05-07'])

dates = pd.to_datetime(date[0], format='%d-%m-%y')
dates[0] + timedelta(days=1)

ticker_df = getTicker(index_df, 'DAX Index')
ticker_date = ticker_df['Date']
pd.to_datetime(ticker_date, format='%Y-%m-%d')

1/3

0.3333333333333333